# Part 2 - VacationPy

## Background
In this project, we will be analyzing data from the google places API in order to create a heatmap and find hotels based on different parameters. We will also mark the locations of these hotels on the heatmap. 

Our first objective is to import our libraries, read the csv, and begin creating the heatmap. 

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import gkey

### Task 1: Reading the CSV from the WeatherPy Notebook and displaying it. 

In [2]:
Datapd = pd.read_csv('output_data/cities.csv')
Datapd.dropna()
Datapd.head()

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temperature,Wind Speed
0,Qaanaaq,0,GL,1584034966,47,77.48,-69.36,-17.32,5.75
1,Mutoko,68,ZW,1584034966,86,-17.40,32.23,68.36,5.73
2,Ancud,90,CL,1584034966,100,-41.87,-73.82,59.00,10.29
3,Tarauacá,15,BR,1584034806,37,-8.16,-70.77,98.96,1.41
4,Busselton,86,AU,1584034966,39,-33.65,115.33,66.00,17.11


### Task 2: Configuring gmaps and Isolating Coordinates and Humidity. 

In [3]:
# Configure gmaps
gmaps.configure(api_key=gkey)

#Seperating Data from Data Frame 
coordinates = Datapd[['Latitude','Longitude']].astype(float)
humidity = Datapd["Humidity"].astype(float)

### Task 3: Building the Heatmap Layer and Adding it. 

In [4]:
#Humidity Heatmap 

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(coordinates, humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Task 4: Specifying my Ideal weather Conditions and Creating a New Data Frame Containing Narrowed Cities.

The parameters I used are 
       
        (1) Max Temperature between 60 and 80 degrees Farenheit. 
        (2) Wind Speed less than 5 mph. 
        (3) Cloudiness less than 1. 

In [5]:
#  Create new DataFrame fitting weather criteria

Max_Temp_Array = Datapd["Max Temperature"].astype(float)
Windspeed_Array = Datapd["Wind Speed"].astype(float)
Cloudiness_Array = Datapd["Cloudiness"].astype(float)
latitudes_Array = Datapd["Latitude"].astype(float)
longitudes_Array = Datapd["Longitude"].astype(float)
cities_array = Datapd["City"]
Country_Array = Datapd["Country"]

latitudes = []
longitudes = []
cities = []
country = []
maxtemp = []
cloudiness = []
windspeed = []


for i in range(len(humidity)):
    if Max_Temp_Array[i] <=80 and Max_Temp_Array[i] >=60 and Windspeed_Array[i] < 5 and Cloudiness_Array[i] < 3:
        latitudes.append(latitudes_Array[i])
        longitudes.append(longitudes_Array[i])
        cities.append(cities_array[i])
        country.append(Country_Array[i])
        maxtemp.append(Max_Temp_Array[i])
        cloudiness.append(Cloudiness_Array[i])
        windspeed.append(Windspeed_Array[i])
   
narrowed_city_df = pd.DataFrame({'City': cities,
                                 'Country' : country,
                                 'Cloudiness': cloudiness,
                                 'Max Temperature' : maxtemp,
                                 'Lat' : latitudes, 
                                 'Lng' : longitudes,
                                 'Wind Speed' : windspeed
                                })

### Task 5: Searching for Hotels within Radius of my Narrowed Cities and Stored the Hotel Results in a Data Frame. 

In [6]:
target_type = "Hotels"
radius = 5000
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
marker_latitude = []
marker_longitude = []
hotel_name = []

for t in range(len(latitudes)): 
    params = {
    "location": f"{latitudes[t]},{longitudes[t]}",
    "types": target_type,
    "radius": radius,
    "key": gkey
    }
    response = requests.get(base_url , params)
    hotels_near = response.json()
    hotel_name.append(hotels_near['results'][0]['name'])
    marker_latitude.append(hotels_near['results'][0]['geometry']['location']['lat'])
    marker_longitude.append(hotels_near['results'][0]['geometry']['location']['lng'])
    
hotelname = pd.DataFrame(hotel_name)    
hotel_df = pd.concat([hotelname, narrowed_city_df],axis=1)
hotel_df.rename(columns = {0 : "Hotel Name"}).head()

,Hotel Name,City,Country,Cloudiness,Max Temperature,Lat,Lng,Wind Speed
0,Lompoc,Lompoc,US,1.0,66.99,34.64,-120.46,4.70
1,Pyin Oo Lwin,Pyn U Lwin,MM,0.0,71.06,22.03,96.47,1.74
2,Alice Springs,Alice Springs,AU,0.0,73.40,-23.70,133.88,4.52
3,Tura,Tura,IN,0.0,73.94,25.52,90.22,3.36
4,Cà Mau,Ca Mau,VN,0.0,75.56,9.18,105.15,3.24


### Task 6: Building Marker Layers and Adding Them to the Heat Map Created Earlier. 

In [7]:
fig1 = gmaps.figure()

narrowed_city_df["Hotel Name"] = hotelname
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
heat_layer1 = gmaps.heatmap_layer(coordinates, humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)
fig1.add_layer(heat_layer1)
fig1.add_layer(markers)
fig1

Figure(layout=FigureLayout(height='420px'))